In [2]:
#============================Import Lib============================

import random
import pandas as pd
import numpy as np
import os
from PIL import Image
import cv2
import matplotlib.pyplot as plt
# from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#============================HyperParam Setting============================
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':3,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':64,
    'SEED':626
}
# 시드 고정 
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정
#============================TRAIN DATA LOAD============================
#============================라벨별로 잘라서 저장("라벨이름_data.csv")============================
train = pd.read_csv('/kaggle/input/dacon00/train.csv')
label_columns = train.columns[2:]
save_directory = '/kaggle/working/'
for label in label_columns:
    # 현재 라벨에 해당하는 데이터만 필터링
    filtered_df = train[label]
    filtered_df = pd.concat([train.iloc[:, 0:2], filtered_df], axis=1)
    # 필터링된 데이터를 별도의 파일로 저장
    # 파일 경로를 포함하여 저장
    filtered_df.to_csv(os.path.join(save_directory, f'train_{label}.csv'), index=False)
# water_data.csv
#============================TEST DATA LOAD============================
#============================라벨별로 잘라서 저장("라벨이름_data.csv")============================
test = pd.read_csv('/kaggle/input/dacon00/test.csv')
# water_data.csv
#============================"라벨이름_data.csv" 불러오기============================
# 라벨 데이터가 있는 폴더 경로
folder_path = '/kaggle/working/'
# 모든 CSV 파일의 리스트를 가져옴
file_list = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 각 파일에 대해 반복
def rewrite(df):
    df['img_path'] = '/kaggle/input/dacon00/' + df['img_path']
    return df
def get_labels(df):
    return df.iloc[:,2:].values
for label in label_columns:
    # CSV 파일 경로
    file_path = os.path.join(folder_path, f'train_{label}.csv')

    # CSV 파일 읽기
    globals()[label] = pd.read_csv(file_path)

    # 이미지 경로 수정
    globals()[label] = rewrite(globals()[label])

    # 훈련 데이터와 검증 데이터로 분할
    train_df, val_df = train_test_split(globals()[label], train_size=0.8, random_state=42)

    # 분할된 데이터셋을 변수에 저장
    globals()[f"train_{label}"] = train_df
    globals()[f"val_{label}"] = val_df

    # 훈련 및 검증 데이터셋에서 라벨 추출
    globals()[f"train_{label}_labels"] = get_labels(train_df)
    globals()[f"val_{label}_labels"] = get_labels(val_df)

    print(f"Train and validation sets with labels for {label} are created.")
    
train_water    
val_water
train_water_labels
val_water_labels

#============================Data Load2============================
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transform=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transform = transform
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        # PIL 이미지로 불러오기
        image = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)
        
        if self.label_list is not None:
            label = torch.tensor(self.label_list[index], dtype=torch.float32)
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])), #,interpolation=transforms.InterpolationMode.LANCZOS
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
#============================TRAIN/VALID============================
train_loaders = {}
val_loaders = {}

for label in label_columns:
    # 훈련 데이터셋 생성
    train_dataset = CustomDataset(
        img_path_list=globals()[f"train_{label}"]['img_path'].tolist(),
        label_list=globals()[f"train_{label}_labels"],
        transform=train_transform
    )

    # 검증 데이터셋 생성
    val_dataset = CustomDataset(
        img_path_list=globals()[f"val_{label}"]['img_path'].tolist(),
        label_list=globals()[f"val_{label}_labels"],
        transform=test_transform
    )

    # 훈련 DataLoader 생성
    train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
    train_loaders[label] = train_loader

    # 검증 DataLoader 생성
    val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    val_loaders[label] = val_loader

    print(f"DataLoaders for training and validation are created for {label}")
train_loaders['trees']
val_loaders['trees']
    
#============================Modeling(ResNet101)============================
class BaseModel(nn.Module):
    def __init__(self, num_classes=1):
        super(BaseModel, self).__init__()
        self.backbone = models.resnet101(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = F.sigmoid(self.classifier(x))
        return x
#============================Training============================
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)

    best_val_loss = float('inf')
    best_model = None

    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            output = model(imgs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        _val_loss = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}]')

        if best_val_loss > _val_loss:
            best_val_loss = _val_loss
            best_model = model

    return best_model

def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            probs = model(imgs)

            loss = criterion(probs, labels)

            val_loss.append(loss.item())

        _val_loss = np.mean(val_loss)

    return _val_loss

##########################
# 'tanks', 'tennis court', 'terrace', 'track', 'trail',
# 'transmission tower', 'trees', 'water', 'wetland', 'wind turbine'


Train and validation sets with labels for airplane are created.
Train and validation sets with labels for airport are created.
Train and validation sets with labels for bare soil are created.
Train and validation sets with labels for baseball diamond are created.
Train and validation sets with labels for basketball court are created.
Train and validation sets with labels for beach are created.
Train and validation sets with labels for bridge are created.
Train and validation sets with labels for buildings are created.
Train and validation sets with labels for cars are created.
Train and validation sets with labels for chaparral are created.
Train and validation sets with labels for cloud are created.
Train and validation sets with labels for containers are created.
Train and validation sets with labels for crosswalk are created.
Train and validation sets with labels for dense residential area are created.
Train and validation sets with labels for desert are created.
Train and validatio

  0%|          | 0/819 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model = BaseModel()
num_gpu = torch.cuda.device_count()  # Dual GPU in KAGGLE
if num_gpu > 1:
    model = nn.DataParallel(model)
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
label_columns
infer_model = train(model, optimizer, train_loaders['tennis court'], val_loaders['tennis court'], device) # 학습 진행

def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

preds = inference(infer_model, test_loader, device)

submit = sample_submission
submit.iloc[:,1:] = preds

submit.to_csv("C:/Users/tkdgn/Desktop/open/tennis court.csv", index = False) 

In [ ]:
model = BaseModel()
num_gpu = torch.cuda.device_count()  # Dual GPU in KAGGLE
if num_gpu > 1:
    model = nn.DataParallel(model)
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
label_columns
infer_model = train(model, optimizer, train_loaders['terrace'], val_loaders['terrace'], device) # 학습 진행


def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

preds = inference(infer_model, test_loader, device)

submit = sample_submission
submit.iloc[:,1:] = preds

submit.to_csv("C:/Users/tkdgn/Desktop/open/terrace.csv", index = False) 


# **#########LABEL 34REAL FIT##############**

# **######자아 돌려보자고~~######**** :: LABEL 34

# #######REALLLLLLLL!!!!!!!!!!!!!!!

In [3]:
#============================Import Lib============================
import random
import pandas as pd
import numpy as np
import os
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#============================HyperParam Setting============================
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':3,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':64,
    'SEED':41
}
# 시드 고정 
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정
#============================TRAIN DATA LOAD============================
#============================라벨별로 잘라서 저장("라벨이름_data.csv")============================
train = pd.read_csv('/kaggle/input/dacon00/train.csv')
label_columns = train.columns[2:]
save_directory = '/kaggle/working/'
for label in label_columns:
    # 현재 라벨에 해당하는 데이터만 필터링
    filtered_df = train[label]
    filtered_df = pd.concat([train.iloc[:, 0:2], filtered_df], axis=1)
    # 필터링된 데이터를 별도의 파일로 저장
    # 파일 경로를 포함하여 저장
    filtered_df.to_csv(os.path.join(save_directory, f'train_{label}.csv'), index=False)
# water_data.csv
#============================TEST DATA LOAD============================
#============================라벨별로 잘라서 저장("라벨이름_data.csv")============================
test = pd.read_csv('/kaggle/input/dacon00/test.csv')
# water_data.csv
#============================"라벨이름_data.csv" 불러오기============================
# 라벨 데이터가 있는 폴더 경로
folder_path = '/kaggle/working/'
# 모든 CSV 파일의 리스트를 가져옴
file_list = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 각 파일에 대해 반복
def rewrite(df):
    df['img_path'] = '/kaggle/input/dacon00/' + df['img_path']
    return df
def get_labels(df):
    return df.iloc[:,2:].values
for label in label_columns:
    # CSV 파일 경로
    file_path = os.path.join(folder_path, f'train_{label}.csv')

    # CSV 파일 읽기
    globals()[label] = pd.read_csv(file_path)

    # 이미지 경로 수정
    globals()[label] = rewrite(globals()[label])

    # 훈련 데이터와 검증 데이터로 분할
    train_df, val_df = train_test_split(globals()[label], train_size=0.8, random_state=42)

    # 분할된 데이터셋을 변수에 저장
    globals()[f"train_{label}"] = train_df
    globals()[f"val_{label}"] = val_df

    # 훈련 및 검증 데이터셋에서 라벨 추출
    globals()[f"train_{label}_labels"] = get_labels(train_df)
    globals()[f"val_{label}_labels"] = get_labels(val_df)

    print(f"Train and validation sets with labels for {label} are created.")
dfs=[train_water, test]
test = [rewrite(df) for df in dfs] 
test = test[1]
train_water    
val_water
train_water_labels
val_water_labels

#============================Data Load2============================

class CustomDataset():
    def __init__(self, img_path_list, label_list, aug_mode='normal', mode='train'):
        self.img_path_list=img_path_list
        self.label_list=label_list
        self.aug_mode=aug_mode
        self.mode=mode
        
        self.kernel_sharpening=np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
        
        if self.aug_mode=='normal' or self.mode=='test':
            self.transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
            
        if self.aug_mode=='rotate':
            self.transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
                transforms.RandomRotation([-180, 180]),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
            
        if self.aug_mode=='flip':
            self.transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
                transforms.RandomHorizontalFlip(p=1),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
            
        
        
    def __getitem__(self, index):
            img_path=self.img_path_list[index]
            
            image=cv2.imread(img_path, cv2.IMREAD_COLOR)
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_LANCZOS4)

            # image=cv2.medianBlur(image,3)
            image=cv2.filter2D(image,-1, self.kernel_sharpening)
            
            
            if self.transform is not None:
                image=self.transform(image)
                
            if self.label_list is not None:
                label = torch.tensor(self.label_list[index], dtype=torch.float32)
                return image,label
            else:
                return image
            
    def __len__(self):
        return len(self.img_path_list)   

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE']),interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE']),interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
#============================TRAIN/VALID============================
train_loaders = {}
val_loaders = {}

for label in label_columns:
    # 훈련 데이터셋 생성
    train_dataset = CustomDataset(
        img_path_list=globals()[f"train_{label}"]['img_path'].tolist(),
        label_list=globals()[f"train_{label}_labels"]
        
    )

    # 검증 데이터셋 생성
    val_dataset = CustomDataset(
        img_path_list=globals()[f"val_{label}"]['img_path'].tolist(),
        label_list=globals()[f"val_{label}_labels"]
       
    )

    # 훈련 DataLoader 생성
    train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
    train_loaders[label] = train_loader

    # 검증 DataLoader 생성
    val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    val_loaders[label] = val_loader

    print(f"DataLoaders for training and validation are created for {label}")
train_loaders['trees']
val_loaders['trees']
    
#============================Modeling(ResNet101)============================
class BaseModel(nn.Module):
    def __init__(self, num_classes=1):
        super(BaseModel, self).__init__()
        self.backbone = models.resnet101(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = F.sigmoid(self.classifier(x))
        return x
#============================Training============================
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)

    best_val_loss = float('inf')
    best_model = None

    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            output = model(imgs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        _val_loss = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}]')

        if best_val_loss > _val_loss:
            best_val_loss = _val_loss
            best_model = model

    return best_model

def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            probs = model(imgs)

            loss = criterion(probs, labels)

            val_loss.append(loss.item())

        _val_loss = np.mean(val_loss)

    return _val_loss

torch.cuda.empty_cache()
##=====================training================================##
##=====================training================================##
##=====================training================================##
##=====================training================================##
##=====================training================================##
##=====================training================================##

Label1 = label_columns[0:10]
Label2 = label_columns[10:20]
Label3 = label_columns[20:30]
Label4 = label_columns[30:40]
Label5 = label_columns[40:50]
Label6 = label_columns[50:60]
print("READY")
best_models = {}

def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

Train and validation sets with labels for airplane are created.
Train and validation sets with labels for airport are created.
Train and validation sets with labels for bare soil are created.
Train and validation sets with labels for baseball diamond are created.
Train and validation sets with labels for basketball court are created.
Train and validation sets with labels for beach are created.
Train and validation sets with labels for bridge are created.
Train and validation sets with labels for buildings are created.
Train and validation sets with labels for cars are created.
Train and validation sets with labels for chaparral are created.
Train and validation sets with labels for cloud are created.
Train and validation sets with labels for containers are created.
Train and validation sets with labels for crosswalk are created.
Train and validation sets with labels for dense residential area are created.
Train and validation sets with labels for desert are created.
Train and validatio

# TRIANING START

# LABEL 3

In [ ]:

for label in Label3:
    print(f"Training model for label: {label}")

    # 각 라벨에 대한 훈련 및 검증 데이터 로더
    train_loader = train_loaders[label]
    val_loader = val_loaders[label]

    # 새 모델 인스턴스 생성
    model = BaseModel()
    num_gpu = torch.cuda.device_count()  # Dual GPU in KAGGLE
    if num_gpu > 1:
        model = nn.DataParallel(model)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=CFG["LEARNING_RATE"])

    # 모델 훈련
    trained_model = train(model, optimizer, train_loader, val_loader, device)

    # 최적의 모델 저장
    best_models[label] = trained_model

    # 모델 저장 (옵션)
    torch.save(trained_model.state_dict(), f'/kaggle/working/best_model_{label}.pth')

test_dataset = CustomDataset(test['img_path'].values, None, test_transform)

def load_model(label):
    model = BaseModel()
    if num_gpu > 1:
        model = nn.DataParallel(model)
    model.load_state_dict(torch.load(f"/kaggle/working/best_model_{label}.pth"))
    model.to(device)
    return model

for label in Label3:
    print(f"Predicting for label: {label}")

    # 테스트 데이터 로더 설정
    test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    # 저장된 모델 불러오기
    model = load_model(label)

    # 추론 실행
    preds = inference(model, test_loader, device)

    # 추론 결과 저장
    sample_submission = pd.read_csv('/kaggle/input/dacon00/sample_submission.csv')
    sample_submission[label] = preds
    sample_submission.to_csv(f"/kaggle/working/{label}.csv", index=False)
    
    

Training model for label: freeway


100%|██████████| 205/205 [03:18<00:00,  1.03it/s]


Epoch [1], Train Loss : [0.04190] Val Loss : [0.02449]


100%|██████████| 205/205 [01:55<00:00,  1.77it/s]


Epoch [2], Train Loss : [0.02071] Val Loss : [0.01796]


...

100%|██████████| 205/205 [01:48<00:00,  1.89it/s]


Epoch [3], Train Loss : [0.03458] Val Loss : [0.03318]
Training model for label: mobile home


100%|██████████| 205/205 [01:51<00:00,  1.85it/s]


Epoch [2], Train Loss : [0.07573] Val Loss : [0.09740]


 59%|█████▉    | 487/819 [07:50<05:18,  1.04it/s]

In [ ]:
Label3=Label3[10]
for label in Label3:
    print(f"Training model for label: {label}")

    # 각 라벨에 대한 훈련 및 검증 데이터 로더
    train_loader = train_loaders[label]
    val_loader = val_loaders[label]

    # 새 모델 인스턴스 생성
    model = BaseModel()
    num_gpu = torch.cuda.device_count()  # Dual GPU in KAGGLE
    if num_gpu > 1:
        model = nn.DataParallel(model)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=CFG["LEARNING_RATE"])

    # 모델 훈련
    trained_model = train(model, optimizer, train_loader, val_loader, device)

    # 최적의 모델 저장
    best_models[label] = trained_model

    # 모델 저장 (옵션)
    torch.save(trained_model.state_dict(), f'/kaggle/working/best_model_{label}.pth')

test_dataset = CustomDataset(test['img_path'].values, None, test_transform)

def load_model(label):
    model = BaseModel()
    if num_gpu > 1:
        model = nn.DataParallel(model)
    model.load_state_dict(torch.load(f"/kaggle/working/best_model_{label}.pth"))
    model.to(device)
    return model

for label in Label3:
    print(f"Predicting for label: {label}")

    # 테스트 데이터 로더 설정
    test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    # 저장된 모델 불러오기
    model = load_model(label)

    # 추론 실행
    preds = inference(model, test_loader, device)

    # 추론 결과 저장
    sample_submission = pd.read_csv('/kaggle/input/dacon00/sample_submission.csv')
    sample_submission[label] = preds
    sample_submission.to_csv(f"/kaggle/working/{label}.csv", index=False)
    
    

# LABEL 4

In [ ]:

##=====================training================================##
##=====================training================================##
##=====================training================================##

for label in Label4:
    print(f"Training model for label: {label}")

    # 각 라벨에 대한 훈련 및 검증 데이터 로더
    train_loader = train_loaders[label]
    val_loader = val_loaders[label]

    # 새 모델 인스턴스 생성
    model = BaseModel()
    num_gpu = torch.cuda.device_count()  # Dual GPU in KAGGLE
    if num_gpu > 1:
        model = nn.DataParallel(model)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=CFG["LEARNING_RATE"])

    # 모델 훈련
    trained_model = train(model, optimizer, train_loader, val_loader, device)

    # 최적의 모델 저장
    best_models[label] = trained_model

    # 모델 저장 (옵션)
    torch.save(trained_model.state_dict(), f'/kaggle/working/best_model_{label}.pth')
    
def load_model(label):
    model = BaseModel()
    if num_gpu > 1:
        model = nn.DataParallel(model)
    model.load_state_dict(torch.load(f"/kaggle/working/best_model_{label}.pth"))
    model.to(device)
    return model

for label in Label4:
    print(f"Predicting for label: {label}")

    # 테스트 데이터 로더 설정
    test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    # 저장된 모델 불러오기
    model = load_model(label)

    # 추론 실행
    preds = inference(model, test_loader, device)

    # 추론 결과 저장
    sample_submission = pd.read_csv('/kaggle/input/vision/sample_submission.csv')
    sample_submission[label] = preds
    sample_submission.to_csv(f"/kaggle/working/{label}.csv", index=False)